<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [3]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [4]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# read api key from txt file
with open('../../data/hopsworks-api-key.txt') as f:
    api_key = f.readline().strip()
    print("Read Hopsworks API key from file")
    os.environ["HOPSWORKS_API_KEY"] = api_key

project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()
location_str

Read Hopsworks API key from file
Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1163417
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "Sweden", "city": "Majorna-Linn\\u00e9", "street": "Masthugget", "aqicn_url": "https://api.waqi.info/feed/A60853/", "latitude": 57.7, "longitude": 11.94}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [ ]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality_v2',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1163417/featurestores/1154120/featuregroups/air_quality). Server response: 
HTTP code: 500, HTTP reason: Internal Server Error, body: b'{"errorCode":270116,"usrMsg":"Error getting feature group table from Hive Metastore: hive.kolumbus_featurestore.air_quality_1 table not found","devMsg":"hive.kolumbus_featurestore.air_quality_1 table not found","errorMsg":"Failed to retrieve feature group meta data"}', error code: 270116, error msg: Failed to retrieve feature group meta data, user msg: Error getting feature group table from Hive Metastore: hive.kolumbus_featurestore.air_quality_1 table not found

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [ ]:
import requests
import pandas as pd


aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df

{'aqi': 7, 'idx': -60853, 'attributions': [{'url': 'https://sensor.community/', 'name': 'Citizen Science project sensor.community', 'station': '13439'}, {'url': 'https://waqi.info/', 'name': 'World Air Quality Index Project'}], 'city': {'geo': [57.698, 11.946], 'name': 'Tredje Långgatan', 'url': 'https://aqicn.org/station/@60853', 'location': 'Göteborgs Mini-Hotel, 31, Tredje Långgatan, Masthugget, Majorna-Linné, Gothenburg, Göteborg, Västra Götaland County, Region Götaland, 41327, Sweden'}, 'dominentpol': 'pm25', 'iaqi': {'pm10': {'v': 2}, 'pm25': {'v': 7}}, 'time': {'s': '2024-11-19 19:31:40', 'tz': '+01:00', 'v': 1732041100, 'iso': '2024-11-19T18:31:40Z'}} data['data']


pm25 country           city      street       date  \
0   7.0  Sweden  Majorna-Linné  Masthugget 2024-11-19   

                                  url  
0  https://api.waqi.info/feed/A60853/

In [ ]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 176.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [ ]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 57.75°N 12.0°E
Elevation 2.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


date  temperature_2m_mean  precipitation_sum  wind_speed_10m_max  \
0 2024-11-19                 4.05                0.0            4.104631   
1 2024-11-20                 0.85                0.0           14.440443   
2 2024-11-21                -0.85                0.0            8.766573   
3 2024-11-22                -3.90                0.0            9.085988   
4 2024-11-23                 3.65                0.0            9.449572   
5 2024-11-24                 1.80                0.7           26.493652   
6 2024-11-25                 8.55                0.6           24.627787   
7 2024-11-26                 8.10                0.9           31.820824   
8 2024-11-27                 4.20                0.0            7.421590   
9 2024-11-28                 6.50                0.0           18.532135   

   wind_direction_10m_dominant           city  
0                   164.744827  Majorna-Linné  
1                     4.289077  Majorna-Linné  
2                   340.820892  Majorna-Linné  
3                   326.309906  Majorna-Linné  
4                   310.364441  Majorna-Linné  
5                   137.202621  Majorna-Linné  
6                   195.255173  Majorna-Linné  
7                   225.458298  Majorna-Linné  
8                   140.906158  Majorna-Linné  
9                   209.054504  Majorna-Linné

In [ ]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 452.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [ ]:
# Retrieve data from the feature store
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)

# Get the data from the feature store
aq_past_df = air_quality_fg.read(read_options={"use_hive": True})

# Generate past dates
past_dates = [today - datetime.timedelta(days=i) for i in range(1, 4)]

# Ensure 'date' column is of type date
aq_past_df['date'] = pd.to_datetime(aq_past_df['date']).dt.date

# Filter the DataFrame for rows that match past dates
aq_past_df_filtered = aq_past_df[aq_past_df['date'].isin(past_dates)]

# Set the mean value for missing historical values
mean_value = 2.620594

# Merge past data with today's data, substituting missing values with the mean
for i, lag in enumerate(['pm25_lag_1', 'pm25_lag_2', 'pm25_lag_3']):
    past_date_data = aq_past_df_filtered[aq_past_df_filtered['date'] == past_dates[i]]['pm25']
    aq_today_df[lag] = past_date_data.values[0] if not past_date_data.empty else mean_value

# Ensure columns are float64
aq_today_df['pm25_lag_1'] = aq_today_df['pm25_lag_1'].astype('float64')
aq_today_df['pm25_lag_2'] = aq_today_df['pm25_lag_2'].astype('float64')
aq_today_df['pm25_lag_3'] = aq_today_df['pm25_lag_3'].astype('float64')


RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1163417/featurestores/1154120/featuregroups/air_quality). Server response: 
HTTP code: 404, HTTP reason: Not Found, body: b'{"errorCode":100048,"usrMsg":"Error: timed out Could not find service ServiceQuery(name=rpc.namenode.service.consul, tags=[])","devMsg":"Error: timed out Could not find service ServiceQuery(name=rpc.namenode.service.consul, tags=[])","errorMsg":"Service not found"}', error code: 100048, error msg: Service not found, user msg: Error: timed out Could not find service ServiceQuery(name=rpc.namenode.service.consul, tags=[])

In [ ]:
# Insert updated data into the feature group
air_quality_fg.insert(aq_today_df)

In [ ]:
# Insert new data
weather_fg.insert(daily_df)

2024-11-19 11:29:18,544 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1163417/fs/1154120/fg/1345925


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1163417/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x2903dd300>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 669723
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-19T10:29:18.000544Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
